<img src = 'fotos/logo_dani.jpeg'>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX 
import itertools
import random
from IPython.display import clear_output
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

In [2]:
def rmse(real, predicted):
    return np.sqrt(((real - predicted) ** 2).mean())

## Ficheros y rutas de entrada/salida 

In [3]:
dir_in = '../../datos/datos_desarrollo'
file1_in = 'consumo_final.csv'
dir_out = dir_in

## Carga de datos 

In [4]:
df_consumo = pd.read_csv(os.path.join(dir_in, file1_in), sep = ';')
df_consumo.columns = [columna.lower() for columna in df_consumo.columns]
df_consumo.rename(columns = {'fecha_inicio': 'mes_inicio_temp', 'fecha_fin': 'mes_fin_temp'}, inplace = True)
df_consumo.date = pd.to_datetime(df_consumo.date, format = '%d/%m/%Y')

In [5]:
precio_model = df_consumo[['ccaa', 'producto', 'volumen_miles_de_kg', 'valor_miles_de_€', 'precio_medio_kg', 'date']]
cmpl_model_dict = {product: {comunidad: precio_model[(precio_model.producto.eq(product))&
                                                      (precio_model.ccaa.eq(comunidad))].drop(['producto', 'ccaa'], axis = 1)
                              for comunidad in precio_model.ccaa.unique()} for product in precio_model.producto.unique()}

In [6]:
df_prueba = df_consumo[(df_consumo.ccaa.isin(['Andalucia', 'Aragon'])) & (df_consumo.producto.isin(['Patatas', 'Mango']))]
prueba_model = df_prueba[['ccaa', 'producto', 'volumen_miles_de_kg', 'valor_miles_de_€', 'precio_medio_kg', 'date']]
sample_model_dict = {product: {comunidad: prueba_model[(prueba_model.producto.eq(product))&
                                                      (prueba_model.ccaa.eq(comunidad))].drop(['producto', 'ccaa'], axis = 1)
                              for comunidad in prueba_model.ccaa.unique()} for product in prueba_model.producto.unique()}

In [7]:
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

In [ ]:
len(pdq) * len(seasonal_pdq)

In [ ]:
def best_sarima(variable):
    try:
        len(predicciones)
    except NameError:
        predicciones = pd.DataFrame(columns = ['ccaa', 'producto'])
    vuelta_general = 1
    for producto, v in cmpl_model_dict.items():
        n_comunidad = 1
        for comunidad in v.keys():
            combinacion = 1
            error_dict = {}
            timeseries_data = cmpl_model_dict[producto][comunidad][[variable, 'date']].copy()
            timeseries_data.index = pd.DatetimeIndex(timeseries_data.date)
            timeseries_data.drop('date', axis = 1, inplace = True)
            timeseries_data.sort_index(inplace = True)

            train = timeseries_data.loc[timeseries_data.index <= '2020-02-01']
            test = timeseries_data.loc[timeseries_data.index > '2020-02-01']
            
            for param in pdq:
                for seasonal_param in seasonal_pdq:
                    print('Tienes', len(cmpl_model_dict), 'productos y vas por el', vuelta_general, '. Este producto tiene', 
                          len(v), 'comunidades y vas por la', n_comunidad, 'Hay un total de', len(pdq) * len(seasonal_pdq), 
                          'combinaciones y vas por la', combinacion, '. La variable es', variable)
                    try:
                        mod = SARIMAX(train, 
                            order = param,
                            seasonal_order = seasonal_param, 
                            enforce_stationarity = False,
                            enforce_invertibility = False)                
                        result = mod.fit()
                        ypred = result.get_forecast(steps = len(test)).predicted_mean.values
                        error = rmse(test.values, ypred)
                        error_dict.update({error: [param, seasonal_param]})
                    except IndexError:
                        continue
                    combinacion += 1
                    clear_output(wait = True)
                    
            min_error = min(list(error_dict.keys()))
            final_model = SARIMAX(train, order = error_dict[min_error][0], seasonal_order = error_dict[min_error][1], 
                                  enforce_stationarity = False, enforce_invertibilty = False)
            final_result = final_model.fit()
            final_ypred = final_result.get_forecast(steps = len(test))
            ypred_df = pd.DataFrame(final_ypred.predicted_mean).rename(columns = {'predicted_mean': str(variable) + '_predicted'})
            pred_ci = final_ypred.conf_int()
            pred_ci_df = pd.concat([ypred_df, pred_ci], axis = 1)
            pred_ci_df[['ccaa', 'producto']] = comunidad, producto
            
            line = pd.to_datetime('2020-02-01', format = '%Y-%m-%d')
            real_value = train.loc[line].values[0]
            new_row =  pd.DataFrame({'ccaa': comunidad, 'producto': producto, (variable + '_predicted'): real_value, 
                         ('lower ' + variable):real_value, ('upper ' + variable): real_value}, 
                                    index = [line])
            predicciones = pd.concat([predicciones, new_row, pred_ci_df], axis = 0)
            n_comunidad += 1
        vuelta_general += 1
    return predicciones

In [ ]:
a = 1
for param in pdq:
    for seasonal_param in seasonal_pdq:
        print([param, seasonal_param])
        a += 1
        clear_output(wait = True)
        if a == 21:
            break

In [ ]:
timeseries_data = sample_model_dict['Patatas']['Andalucia'][['volumen_miles_de_kg', 'date']].copy()
timeseries_data.index = pd.DatetimeIndex(timeseries_data.date)
timeseries_data.drop('date', axis = 1, inplace = True)
timeseries_data.sort_index(inplace = True)

train = timeseries_data.loc[timeseries_data.index <= '2020-02-01']
test = timeseries_data.loc[timeseries_data.index > '2020-02-01']

In [ ]:
mod = SARIMAX(train, 
        order = param,
        seasonal_order = seasonal_param, 
        enforce_stationarity = False,
        enforce_invertibility = False)

In [ ]:
df_peque = best_sarima('volumen_miles_de_kg')
df_medi = pd.concat([df_peque, best_sarima('valor_miles_de_€')], axis = 1)
df_gran = pd.concat([df_medi, best_sarima('precio_medio_kg')], axis = 1)

In [ ]:
df_final = df_gran.T.drop_duplicates().T
df_final.to_csv(os.path.join(dir_out, 'predicciones_consumo.csv'), sep = ';')